<a href="https://colab.research.google.com/github/ryannhally/Comps_Project_and_Essay/blob/main/CS_490_Train_w_no_normalizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate sacrebleu elotl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
from datasets import load_dataset
from datasets import Dataset

In [ ]:
import elotl.corpus
import elotl.nahuatl.orthography

axolotl = elotl.corpus.load('axolotl')
bible_raw = load_dataset('somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl')
bible_filter = bible_raw.filter(lambda x:x['sp'] is not None)
bible = bible_filter.filter(lambda x:x['src'] == 'opus-bible-uedin' )



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/6.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20024 [00:00<?, ? examples/s]

In [ ]:
print(bible['train'][0])

{'Unnamed: 0': 0, 'sp': ' Libro de la genealogía de Jesucristo, hijo de David, hijo de Abraham', 'nah': 'In se tocayomej de itajhuehuentzitzihuan Jesucristo yejhuan teicniu itech David niman Abraham. ', 'var': None, 'src': 'opus-bible-uedin'}


In [ ]:
# Normalize
n = elotl.nahuatl.orthography.Normalizer("sep")

translation = []
idx = []
for i, line in enumerate(axolotl):
    idx.append(i)
    translation.append({'es': line[0], 'na': n.normalize(line[1])})

index = i


In [ ]:
translation[0], idx[0]

({'es': 'Vino a iluminar el sol y allí fue a ver a su',
  'na': 'tlaminako tonati uan noponi kiitato'},
 0)

In [ ]:
print(index)

16116


In [ ]:
# Normalize
i = 0
for example in bible['train']:
  index +=1
  idx.append(index)
  translation.append({'es': bible['train'][i]['sp'], 'na': n.normalize(bible['train'][i]['nah'])})
  i +=1

In [ ]:
print(index)
print(i)

23937
7821


In [ ]:
print(translation[0])
print(translation[3000])
print(translation[9000])
print(translation[1400])

{'es': 'Vino a iluminar el sol y allí fue a ver a su', 'na': 'tlaminako tonati uan noponi kiitato'}
{'es': 'En este año se enseñoreó el señor Coxcoxtli como tlatohuani de Colhuacan; este Coxcoxtli era suegro de los tenancas atlauhtecas, porque la señora que mandó pedir y tomó por esposa Ilancuéitl Atlauhtecatlteuctli era su nieta.', 'na': 'ipan in motlajtokatlali in tlakatl koxkoxtli tlajtouani kuluakan; inuexiu in tenanka atlauteka || 127r inin koxkoxtli ka ixuiutsin in siuapili i konan i komitlani in ilankueitl atlautekatlteuktli.'}
{'es': 'Usos:', 'na': 'kualtia'}
{'es': 'Luego partieron los que llevan a cuestas a su dios,', 'na': 'niman ik yake in teumamake,'}


In [ ]:
my_dict = { "id": idx, "translation": translation}

books = Dataset.from_dict(my_dict)
books = books.train_test_split(test_size=0.1)

In [ ]:
books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 21544
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 2394
    })
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
source_lang = "es"

target_lang = "na"

prefix = "translate Spanish to Nahuatl: "


def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples["translation"]]

    targets = [example[target_lang] for example in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():

        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/21545 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2394 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# Sign into HuggingFace
from google.colab import userdata
userdata.get('HuggingFaceKey')

from huggingface_hub import notebook_login

notebook_login('HuggingFaceKey')

In [ ]:
training_args = Seq2SeqTrainingArguments(

    output_dir="ryannhally/t5-small-nah-tutorial2",

    evaluation_strategy="steps",

    eval_steps=100,

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    save_total_limit=3,

    num_train_epochs=3,

    fp16=True,

    push_to_hub=True

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_books["train"],

    eval_dataset=tokenized_books["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

<ipython-input-24-7c93e7e4f211>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
1000,3.411000,3.035071
2000,3.076000,2.766193
3000,2.899700,2.614539
4000,2.772200,2.515382
5000,2.693900,2.434111
6000,2.619400,2.374817
7000,2.561800,2.324315
8000,2.515500,2.276291
9000,2.478100,2.240551
10000,2.426200,2.204147


In [ ]:
model.to("cpu")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Save model and tokenizer
trainer.model.save_pretrained('ryannhally/t5-small-nah-tutorial2')
tokenizer.save_pretrained('ryannhally/t5-small-nah-tutorial2')

('ryannhally/t5-small-nah-tutorial2/tokenizer_config.json',
 'ryannhally/t5-small-nah-tutorial2/special_tokens_map.json',
 'ryannhally/t5-small-nah-tutorial2/spiece.model',
 'ryannhally/t5-small-nah-tutorial2/added_tokens.json',
 'ryannhally/t5-small-nah-tutorial2/tokenizer.json')